In [123]:
import pandas as pd

In [201]:
data = pd.read_csv("dataset.csv", delimiter=';')

In [202]:
train_data = []

In [203]:
for _, row in data.iterrows():
    text = row['text']
    origin = row['origin']
    destination = row['destination']
    detours = row['detours'] if pd.notna(row['detours']) else ""
    
    # Vérifier que le texte est en français et que c'est un itinéraire valide
    if origin != "NOT_FRENCH" and origin != "NOT_TRIP":
        # Trouver les positions d'origine et de destination dans le texte
        start_origin = text.lower().find(origin)
        end_origin = start_origin + len(origin)
        start_destination = text.lower().find(destination)
        end_destination = start_destination + len(destination)
        
        # Liste temporaire pour les positions uniques de détour
        detour_positions = []
        
        if detours:
            for detour in detours.split(","):
                detour = detour.strip()
                start_detour = text.lower().find(detour)
                end_detour = start_detour + len(detour)
                
                # Vérifier que le détour ne chevauche pas ORIGIN ou DESTINATION
                if start_detour >= 0 and (
                    end_detour <= start_origin or start_detour >= end_origin
                ) and (
                    end_detour <= start_destination or start_detour >= end_destination
                ):
                    # Vérifier qu'il n'y a pas de chevauchement avec d'autres détours déjà ajoutés
                    overlap = any(
                        (start < end_detour and end > start_detour)
                        for start, end, _ in detour_positions
                    )
                    if not overlap:
                        detour_positions.append((start_detour, end_detour, "DETOUR"))
        
        # Ajouter les annotations d'origine, destination et détours sans chevauchements
        if start_origin >= 0 and start_destination >= 0 and end_origin <= start_destination:
            entities = [
                (start_origin, end_origin, "ORIGIN"),
                (start_destination, end_destination, "DESTINATION")
            ] + detour_positions  # Ajouter les détours non-chevauchants

            train_data.append((text, {"entities": entities}))

In [127]:
import spacy
from spacy.training.example import Example
import random

In [204]:
nlp = spacy.blank("fr")

In [205]:
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

In [206]:
ner.add_label("ORIGIN")
ner.add_label("DESTINATION")
ner.add_label("DETOUR")

1

In [207]:
optimizer = nlp.begin_training()

In [208]:
for itn in range(5):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], losses=losses, drop=0.5, sgd=optimizer)
    print(f"Itération {itn} - Losses: {losses}")


/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Je veux voyager à autruy sur juine de ger" with entities "[(12, 15, 'ORIGIN'), (18, 34, 'DESTINATION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "En passant par moncel sur vair, st paul en cornill..." with entities "[(114, 117, 'ORIGIN'), (127, 133, 'DESTINATION'), ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/gurkan/coding/python/tokenizer/.venv/lib/python3.9/site-packages/spac

Itération 0 - Losses: {'ner': 4299.90973671333}
Itération 1 - Losses: {'ner': 985.861847242741}
Itération 2 - Losses: {'ner': 935.811436770639}
Itération 3 - Losses: {'ner': 722.0249990224086}
Itération 4 - Losses: {'ner': 708.8160397640829}


In [139]:
nlp.to_disk("model_trajectoire_detour")

In [163]:
from langdetect import detect, LangDetectException

In [164]:
def is_french(text):
    try:
        return detect(text) == "fr"
    except LangDetectException:
        return False

In [165]:
def test_trip_request(text):
    if not is_french(text):
        return "NOT_FRENCH"
    
    doc = nlp(text)
    origin, destination = None, None
    detours = []

    for ent in doc.ents:
        if ent.label_ == "ORIGIN":
            origin = ent.text
        elif ent.label_ == "DESTINATION":
            destination = ent.text
        elif ent.label_ == "DETOUR":
            detours.append(ent.text)
    
    if origin and destination:
        return (text, origin, destination, detours)
    else:
        return "NOT_TRIP"

In [166]:
test_phrases = [
    "Je veux aller de paris à lyon",
    "J'aimerai aller de lille à nice",
    "Voyage de rouen jusqu'à nice",
    "Quel est le trajet de toulouse à bordeaux ?",
    "Je veux aller de paris à lyon en passant par nice",
    "En passant par toulouse, je veux aller de paris à lyon",
    "J'aime bien les restaurants de paris",
    "What time is it in Paris ?",
    "Quel est le trajet de strasbourg à bordeaux ?",
    "Quel est le trajet de bordeaux à strasbourg en passant par lyon ?",
    "Comment me rendre à strasbourg depuis nice ?",

]

In [209]:
for phrase in test_phrases:
    print(f"sentence: {phrase}")
    print(test_trip_request(phrase))


sentence: Je veux aller de paris à lyon
('Je veux aller de paris à lyon', 'paris', 'lyon', [])
sentence: J'aimerai aller de lille à nice
("J'aimerai aller de lille à nice", 'lille', 'nice', [])
sentence: Voyage de rouen jusqu'à nice
("Voyage de rouen jusqu'à nice", 'rouen', 'nice', [])
sentence: Quel est le trajet de toulouse à bordeaux ?
('Quel est le trajet de toulouse à bordeaux ?', 'toulouse', 'bordeaux', [])
sentence: Je veux aller de paris à lyon en passant par nice
('Je veux aller de paris à lyon en passant par nice', 'paris', 'lyon', ['nice'])
sentence: En passant par toulouse, je veux aller de paris à lyon
('En passant par toulouse, je veux aller de paris à lyon', 'paris', 'lyon', ['toulouse'])
sentence: J'aime bien les restaurants de paris
NOT_TRIP
sentence: What time is it in Paris ?
NOT_FRENCH
sentence: Quel est le trajet de strasbourg à bordeaux ?
('Quel est le trajet de strasbourg à bordeaux ?', 'strasbourg', 'bordeaux', [])
sentence: Quel est le trajet de bordeaux à stra

In [214]:
test_trip_request("Je suis à parcey et je veux aller à lyon")

('Je suis à parcey et je veux aller à lyon', 'suis', 'lyon', [])